### Torrent to Google Drive Downloader - Seedr Style

#### ==== USER CONFIGURATION ====
##### Examples:
 - .torrent file URL: "https://yts.mx/torrent/download/xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
 - Magnet link: "magnet:?xt=urn:btih:xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [ ]:
TORRENT_LINK = "https://yts.mx/torrent/download/A87D36A8EFD2924E9FB607722DE5B453C625884A"

In [ ]:
while True:
    TORRENT_LINK = input("Enter Torrent/Magnet Link Or Type Exit: ")
    if magnet_link.lower() == "exit":
        break

In [ ]:
# ==== INSTALL DEPENDENCIES ====
!apt-get install -y python3-libtorrent
!pip install -q ipywidgets requests

# ==== MOUNT GOOGLE DRIVE ====
from google.colab import drive
drive.mount("/content/drive")

# ==== AUTO-RESTART RUNTIME ====
import os
os.kill(os.getpid(), 9)

In [2]:
import libtorrent as lt
import time, os, requests
import ipywidgets as widgets
from IPython.display import display

# ==== SETTINGS ====
BASE_PATH = "/content/drive/My Drive/Torrent"
os.makedirs(BASE_PATH, exist_ok=True)

# ==== INPUT LINK ====
TORRENT_LINK = input("📥 Enter Torrent/Magnet Link: ").strip()

# ==== INIT LIBTORRENT ====
ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()

downloads = []

def add_torrent_from_file(file_path):
    info = lt.torrent_info(file_path)
    save_path = f"{BASE_PATH}/{info.name()}"
    os.makedirs(save_path, exist_ok=True)
    params = {"save_path": save_path, "ti": info}
    downloads.append(ses.add_torrent(params))
    print(f"Added: {info.name()}")

def add_torrent_from_url(url):
    local_file = "/content/temp.torrent"
    r = requests.get(url, allow_redirects=True)
    open(local_file, "wb").write(r.content)
    add_torrent_from_file(local_file)

def add_torrent_from_magnet(link):
    params = {"save_path": BASE_PATH}
    h = lt.add_magnet_uri(ses, link, params)
    downloads.append(h)
    print("Magnet link added")

# ==== DETECT TYPE ====
if TORRENT_LINK.startswith("magnet:"):
    add_torrent_from_magnet(TORRENT_LINK)
elif TORRENT_LINK.endswith(".torrent") or "yts.mx/torrent/download" in TORRENT_LINK:
    add_torrent_from_url(TORRENT_LINK)
else:
    raise ValueError("❌ Invalid link.")

# ==== PROGRESS BARS ====
state_str = [
    "queued", "checking", "downloading metadata",
    "downloading", "finished", "seeding",
    "allocating", "checking fastresume"
]
bars = [widgets.FloatSlider(step=0.01, disabled=True, layout=widgets.Layout(width="auto"))
        for _ in downloads]
display(*bars)

while downloads:
    shift = 0
    for idx, dl in enumerate(downloads[:]):
        bar = bars[idx + shift]
        if not dl.is_seed():
            s = dl.status()
            eta = (s.total_wanted - s.total_done) / s.download_rate if s.download_rate > 0 else 0
            bar.description = f"{dl.name()} | {s.download_rate/1000:.1f} kB/s | {s.progress*100:.1f}% | ETA: {eta/60:.1f} min | {state_str[s.state]}"
            bar.value = s.progress * 100
        else:
            shift -= 1
            ses.remove_torrent(dl)
            downloads.remove(dl)
            bar.close()
            bars.remove(bar)
            print(f"✅ {dl.name()} complete")
    time.sleep(1)

# ==== UNMOUNT DRIVE ====
drive.flush_and_unmount()
print("🎉 All torrents done. Drive unmounted.")


ModuleNotFoundError: No module named 'libtorrent'